In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# import data
hprc_pacbio_hifi = pd.read_csv('/private/groups/hprc/human-pangenomics/documentation-metadata/HPRC_metadata/data/hprc-data-explorer-tables/HPRC_PacBio_HiFi.tsv',sep='\t')

In [3]:
# WUSTL HPRC HiFi Y2 TopUp
sra_identifiers = pd.read_csv('/private/groups/hprc/human-pangenomics/documentation-metadata/HPRC_metadata/submissions/WUSTL_HPRC_HiFi_Year2_TopUp/metadata-14704558-processed-ok.tsv',sep='\t')

post_sra = pd.read_csv('/private/groups/hprc/human-pangenomics/documentation-metadata/HPRC_metadata/submissions/WUSTL_HPRC_HiFi_Year2_TopUp/sra_biosample_WUSTL_Y2_TopUp.tsv',sep='\t')

post_qc = pd.read_csv('/private/groups/hprc/human-pangenomics/documentation-metadata/HPRC_metadata/submissions/WUSTL_HPRC_HiFi_Year2_TopUp/WUSTL_HPRC_HiFi_Year2_TopUp_sample_file_hifi_qc.csv')

post_qc.drop(columns=['filename'],inplace=True)
post_qc.rename(columns={'bam_file':'filename'},inplace=True)

metadata = pd.read_csv('/private/groups/hprc/human-pangenomics/documentation-metadata/HPRC_metadata/submissions/WUSTL_HPRC_HiFi_Year2_TopUp/HPRC_WUSTL_Y2_PacBio_HiFi_TopUp_Metadata_Submission_v0.2.tsv',sep='\t')

release = pd.read_csv('/private/groups/hprc/human-pangenomics/documentation-metadata/HPRC_metadata/submissions/WUSTL_HPRC_HiFi_Year2_TopUp/WUSTL_HiFi_Year2_TopUp.transfer.csv',header=None)
release['filename'] = [filename.split('/')[-1] for filename in release[1].tolist()]
release.rename(columns={1:'path'},inplace=True)

In [4]:
data_table = pd.merge(post_qc, post_sra, on='filename')

In [5]:
assert data_table.shape[0] == post_qc.shape[0] == post_sra.shape[0]

In [12]:
data_table = pd.merge(data_table, 
					  release[['filename','path']], on='filename', how='inner')

data_table['data_type'] = 'unaligned reads'
data_table['DeepConsensus_version'] = np.nan
data_table['notes'] = np.nan

data_table = pd.merge(sra_identifiers[['filename','accession','study']], data_table, on='filename', how='inner')

In [17]:
data_table['sample_ID'] = [sample_id.split('_')[0] for sample_id in data_table['library_ID'].tolist()]

In [18]:
data_table = data_table[list(hprc_pacbio_hifi)]

In [21]:
# List of example columns (from the metadata sheet)

# Check for missing columns in the new data
missing_columns = [col for col in list(hprc_pacbio_hifi) if col not in data_table.columns]

# Check for extra columns in the new data
extra_columns = [col for col in data_table.columns if col not in list(hprc_pacbio_hifi)]

assert missing_columns == extra_columns

In [23]:
data_table.to_csv('WUSTL_HiFi_Y2_TopUp_final_table.csv',index=False)

In [24]:
hprc_pacbio_hifi = pd.concat([hprc_pacbio_hifi, data_table])

In [ ]:
hprc_pacbio_hifi.to_csv('/private/groups/hprc/human-pangenomics/documentation-metadata/HPRC_metadata/data/hprc-data-explorer-tables/HPRC_PacBio_HiFi.tsv',sep='\t', index=False)